In [1]:
import nltk
import pandas as pd

In [40]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('brown')
# nltk.download('tagsets')
# nltk.download('gutenberg')
# nltk.download('genesis')
# nltk.download('inaugural')
# nltk.download('nps_chat')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\princ\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data]

True

In [2]:
file = open('nltk.txt')
read_file = file.read()
text = nltk.Text(nltk.word_tokenize(read_file))
tags = nltk.pos_tag(text)
words = [word for (word, pos) in nltk.pos_tag(text)]
pos = [pos for (word, pos) in nltk.pos_tag(text)]
# match = text.concordance('car')

In [3]:
prh = {'Words':words,'Pos':pos}
df = pd.DataFrame(prh)
df

,Words,Pos
0,The,DT
1,is,VBZ
2,a,DT
3,car,NN
4,.,.
5,Will,NNP
6,is,VBZ
7,running,VBG
8,.,.
9,The,DT


In [33]:
word1 = "The"
word2 = "NN"
res = df[(df['Words'].str.contains(word1)) | (df['Pos'].str.contains(word1))]

In [34]:
res['index'] = res.index
res

<ipython-input-34-fe863e7e6427>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['index'] = res.index


,Words,Pos,index
0,The,DT,0
9,The,DT,9


In [35]:
# first filter passed
indices = res.iloc[:,2].values
indices = indices + 1
cnd2 = df.loc[indices]
cnd2

,Words,Pos
1,is,VBZ
10,car,NN


In [36]:
resFin = cnd2[(cnd2['Words'].str.contains(word2)) | (cnd2['Pos'].str.contains(word2))]

In [37]:
resFin

,Words,Pos
10,car,NN
